# IO

## 1. Paths

In [1]:
const path = require('path');

### 1.1. Normalize

In [30]:
{
    const dir = 'a/b/c/../d';
    
    const normalizedDir = path.normalize(dir);
    console.log(`* when path is "${dir}", then path.normalize(dir) is: "${normalizedDir}"`);
}

* when path is "a/b/c/../d", then path.normalize(dir) is: "a/b/d"


### 1.2. Join

In [31]:
{
    const dir1 = 'a/b/c';
    const dir2 = '../x/y';
    const fn = 'foo.txt';
    
    const joinedPath = path.join(dir1, dir2, fn);
    console.log(`* when "dir1=${dir1}", "dir2=${dir2}", "fn=${fn}" then "path.join(dir1, dir2, fn)" is: "${joinedPath}"`);
}

* when "dir1=a/b/c", "dir2=../x/y", "fn=foo.txt" then "path.join(dir1, dir2, fn)" is: "a/b/x/y/foo.txt"


### 1.3. Absolute path

In [32]:
{
    const dir = '../a/b/c';
    
    const absPath = path.resolve(dir);
    console.log(`* when path is "${dir}", then path.resolve(dir) is: "${absPath}"`);
}

* when path is "../a/b/c", then path.resolve(dir) is: "/Users/alvin/Workspace/Study/study-node/basic/a/b/c"


### 1.4. Relative path

In [33]:
{
    const dir1 = 'a/b/c/d/';
    const dir2 = 'a/b/d/';
    
    const re = path.relative(dir1, dir2);
    console.log(`* relative path of "${dir1}" and "${dir2}" is: "${re}"`);
}

* relative path of "a/b/c/d/" and "a/b/d/" is: "../../d"


### 1.5. Name of directory

In [34]:
{
    const fullpath = 'a/b/c/d.txt';
    
    const dirname = path.dirname(fullpath);
    console.log(`* dirname of "${fullpath}" is: "${dirname}"`);
}

* dirname of "a/b/c/d.txt" is: "a/b/c"


### 1.6. File name of path

In [35]:
{
    const fullpath = 'a/b/c/d.txt';
    
    const basename = path.basename(fullpath);
    console.log(`* basename of "${fullpath}" is: "${basename}"`);
}

* basename of "a/b/c/d.txt" is: "d.txt"


### 1.7. File extenstion name

In [36]:
{
    const fullpath = 'a/b/c/d.txt';
    
    const extname = path.extname(fullpath);
    console.log(`* extname of "${fullpath}" is: "${extname}"`);
}

* extname of "a/b/c/d.txt" is: ".txt"


## 2. File operator

In [37]:
const fs = require('fs');

### 2.1. Remove file

- Function

In [38]:
async function rm(filename) {
    return await fs.promises.unlink(filename);
}

### 2.2. Is file exist

- Function

In [39]:
async function exist(filename) {
    try {
        await fs.promises.access(filename);
        return true;
    } catch (err) {
        return false;
    }
}

- Test

In [40]:
{
    let dir = 'io.ipynb';
    let ok = await exist('io.ipynb');
    console.log(`* "${dir}" file exists? ${ok}`);
    
    dir = path.resolve('../src');
    ok = await exist(dir);
    console.log(`* "${dir}" directory exists? ${ok}`);
}

* "io.ipynb" file exists? true
* "/Users/alvin/Workspace/Study/study-node/basic/src" directory exists? false


### 2.3. Create empty file

- Function

In [41]:
function touch(filename) {
    return new Promise((resolve, reject) => {
        fs.promises.open(filename, 'w')
            .then(h => fs.close(h.fd, () => resolve()))
            .catch(err => reject(err));
    });
}

- Test

In [42]:
{
    try {
        await touch('test.txt');
        const ok = await exist('test.txt');
        console.log(`* after file created, "test.txt" exist? ${ok}`);
    } finally {
        await rm('test.txt');
    }
}

* after file created, "test.txt" exist? true


### 2.4. Create or remove empty folder

- Functions

In [43]:
async function mkdir(dirname, options = {}) {
    try {
        return await fs.promises.mkdir(dirname, options);
    } catch (err) {
        console.log(err.message);
    }
}

async function rmdir(dirname, options = {}) {
    try {
        return await fs.promises.rmdir(dirname, options);
    } catch (err) {
        console.log(err.message);
    }
}

- Test

In [44]:
{
    await mkdir('test');
    
    const ok = await exist('test');
    console.log(`* after directory created, "test" exist? ${ok}`);

    await rmdir('test');
}

UncaughtException: [Error: EBADF: Closing file descriptor 53 on garbage collection failed, close] {
  errno: -9,
  code: 'EBADF',
  syscall: 'close'
}


* after directory created, "test" exist? true


### 3.5. Remove dirs

- Functions

In [45]:
const fse = require('fs-extra');

function mkdirs(pathname) {
    return new Promise((resolve, reject) => {
        fse.mkdirs(pathname, err => {
            if (err) {
                reject(err);
            } else {
                resolve();
            }
        });
    });
}

function rmdirs(pathname) {
    return new Promise((resolve, reject) => {
        fse.remove(pathname, err => {
            if (err) {
                reject(err);
            } else {
                resolve();
            }
        });
    });
}

- Test

In [46]:
{
    const dir = 'test/a/b';
    const file = path.join(dir, 'c.txt');
    
    await mkdirs(dir);
    await touch(file);
    
    const ok = await exist(file);
    console.log(`* file "${file}" exist? ${ok}`);
    
    await rmdirs('test');
}

* file "test/a/b/c.txt" exist? true


## 3. Buffer

### 3.1. Convert with string

In [47]:
{
    const strSrc = 'Hello 大家好';
    
    const bufSrc = Buffer.from(strSrc, 'utf-8');
    const data = [...bufSrc];
    console.log(`* convert string "${strSrc}" to bytes are: ${JSON.stringify(data)}`);
    
    const bufDst = Buffer.from(data);
    const strDst = bufDst.toString('utf-8');
    console.log(`* convert data "${JSON.stringify(data)}" to string is: "${strDst}"`);
}

* convert string "Hello 大家好" to bytes are: [72,101,108,108,111,32,229,164,167,229,174,182,229,165,189]
* convert data "[72,101,108,108,111,32,229,164,167,229,174,182,229,165,189]" to string is: "Hello 大家好"


UncaughtException: [Error: EBADF: Closing file descriptor 53 on garbage collection failed, close] {
  errno: -9,
  code: 'EBADF',
  syscall: 'close'
}


### 3.2. Concat

In [48]:
{
    const str1 = 'Hello ';
    const str2 = '大家好';
    
    const buf1 = Buffer.from(str1, 'utf-8');
    const buf2 = Buffer.from(str2, 'utf-8');
    
    const bufAll = Buffer.concat([buf1, buf2]);
    const data = [...bufAll];
    
    console.log(`* concat "${JSON.stringify([...buf1])}" and "${JSON.stringify([...buf2])}" is: ${JSON.stringify(data)}`);
    
    const strAll = Buffer.from(data).toString('utf-8');
    console.log(`* convert data "${JSON.stringify(data)}" to string is: "${strAll}"`);
}

* concat "[72,101,108,108,111,32]" and "[229,164,167,229,174,182,229,165,189]" is: [72,101,108,108,111,32,229,164,167,229,174,182,229,165,189]
* convert data "[72,101,108,108,111,32,229,164,167,229,174,182,229,165,189]" to string is: "Hello 大家好"


### 3.3. Read write data

In [49]:
const crypto = require('crypto');

{
    let data = null;
    {
        const str = 'hello 大家好';
        const sbuf = Buffer.from(str, 'utf-8');
        const lbuf = Buffer.alloc(4);
        lbuf.writeInt32BE(sbuf.byteLength, 0);
        const cbuf = crypto.createHash('md5').update(sbuf).digest();
        
        data = Buffer.concat([lbuf, sbuf, cbuf]);
        console.log(`* the encoding data is: "${data.toString('hex')}"`);
    }
    
    {
        const len = data.readInt32BE(0);
        console.log(`* the data length is: ${len}`);

        const sbuf = data.slice(4, 4 + len);
        const cbuf = data.slice(4 + len);
        
        const str = sbuf.toString('utf-8');
        console.log(`* the message string in buffer is: "${str}"`);
        
        const checksum = crypto.createHash('md5').update(sbuf).digest();
        console.log(`* calculate checksum is: ${checksum.toString('hex')} and validate checksum is: ${cbuf.toString('hex')}`);
    }
}

* the encoding data is: "0000000f68656c6c6f20e5a4a7e5aeb6e5a5bd5e5d25ed2640de1c7cf160dc56fb9ecf"
* the data length is: 15
* the message string in buffer is: "hello 大家好"
* calculate checksum is: 5e5d25ed2640de1c7cf160dc56fb9ecf and validate checksum is: 5e5d25ed2640de1c7cf160dc56fb9ecf


### 3.4. Base64

In [50]:
{
    let base64 = '';
    
    {
        const str = 'hello 大家好';
        const buf = Buffer.from(str, 'utf-8');
        
        base64 = buf.toString('base64');
        console.log(`* string ${str} to base64 is: "${base64}"`);
    }
    
    {
        const buf = Buffer.from(base64, 'base64');
        const str = buf.toString('utf-8');
        console.log(`* base64 "${base64}" to string is: "${str}"`);
    }
}

* string hello 大家好 to base64 is: "aGVsbG8g5aSn5a625aW9"
* base64 "aGVsbG8g5aSn5a625aW9" to string is: "hello 大家好"


## 4. File read and write

### 4.1. Read and write file

- Function

In [51]:
async function fwrite(filename, data) {
    return await fs.promises.writeFile(filename, data);
}

async function fappend(filename, data) {
    return await fs.promises.appendFile(filename, data);
}

async function fread(filename) {
    return await fs.promises.readFile(filename);
}

async function freadString(filename, encoding='utf-8') {
    return await fs.promises.readFile(filename, encoding);
}

14:49 - No overload matches this call.
14:49 - Overload 1 of 3, '(path: string | Buffer | URL | FileHandle, options?: { encoding?: null; flag?: OpenMode; }): Promise<Buffer>', gave the following error.
14:49 - Type 'string' has no properties in common with type '{ encoding?: null; flag?: OpenMode; }'.
14:49 - Overload 2 of 3, '(path: string | Buffer | URL | FileHandle, options: { encoding: BufferEncoding; flag?: OpenMode; } | "ascii" | "utf8" | "utf-8" | "utf16le" | ... 5 more ... | "hex"): Promise<...>', gave the following error.
14:49 - Argument of type 'string' is not assignable to parameter of type '{ encoding: BufferEncoding; flag?: OpenMode; } | "ascii" | "utf8" | "utf-8" | "utf16le" | "ucs2" | "ucs-2" | "base64" | "latin1" | "binary" | "hex"'.
14:49 - Overload 3 of 3, '(path: string | Buffer | URL | FileHandle, options?: "ascii" | "utf8" | "utf-8" | "utf16le" | "ucs2" | "ucs-2" | "base64" | "latin1" | "binary" | "hex" | (BaseEncodingOptions & { ...; })): Promise<...>', gave the 

- Test

In [ ]:
{
    const filename = 'test.txt';
    
    try {
        await fwrite(filename, Buffer.from('Hello', 'utf-8'));
        await fappend(filename, Buffer.from(' 大家好', 'utf-8'));
        
        const data = await fread(filename);
        console.log(`* read from file: "${data.toString('hex')}" and content is: "${data.toString('utf-8')}"`);
        
        const str = await freadString(filename);
        console.log(`* read from file: "${str}"`);
    } finally {
        await rm(filename);
    }
}

### 4.2. Read and write file from fd

- Functions

In [ ]:
async function lfopen(filename, mode='w+') {
    return await fs.promises.open(filename, mode);
}

function lfclose(fd) {
    return new Promise((resolve, reject) => {
        fs.close(fd, err => {
            if (err) {
                reject(err);
            } else {
                resolve();
            }
        });
    });
}

function lfwrite(fd, buf) {
    return new Promise((resolve, reject) => {
         fs.write(fd, buf, (err, written) => {
             if (err) {
                 reject(err);
             } else {
                 resolve(written);
             }
         });
    });
}

function lfread(fd, buf, offset=0, length=-1, position=0) {
    if (length < 0) {
        length = buf.byteLength;
    }
    return new Promise((resolve, reject) => {
        fs.read(fd, buf, offset, length, position, (err, bytesRead) => {
            if (err) {
                reject(err);
            } else {
                resolve(bytesRead);
            }
        });
    });
}

- Test

In [ ]:
{
    const filename = 'test.txt';
    
    try {
        const wbuf = Buffer.from('Hello 大家好', 'utf-8');

        const wfh = await lfopen(filename);
        const wlen = await lfwrite(wfh.fd, wbuf);
        console.log(`* file written, ${wlen} bytes was written`);
        lfclose(wfh.fd);
        
        const rfh = await lfopen(filename, 'r');
        const rbuf = Buffer.alloc(256);
        const rlen = await lfread(rfh.fd, rbuf);
        console.log(`* file read, ${rlen} bytes was read`);
        lfclose(rfh.fd);
        
        console.log(`* read content is: ${rbuf.toString('utf-8')}`);
    } finally {
        await rm(filename);
    }
}

### 4.3. Read and write from stream

- Functions

In [29]:
function swrite(filename, contents, encoding='utf-8') {
    return new Promise((resolve, reject) => {
        const ws = fs.createWriteStream(filename);
        console.log(`* write stream open? ${ws.writable}`);

        ws.setDefaultEncoding(encoding);
        
        ws.on('error', err => reject(err));
        ws.on('finish', () => {
            resolve(ws.bytesWritten);
            ws.close();
        });

        for (let content of contents) {
            ws.write(content);
        }
        ws.end();
    });
}

function sread(filename, encoding='utf-8') {
    return new Promise((resolve, reject) => {
        const ws = fs.createReadStream(filename);
        console.log(`* read stream open? ${ws.readable}`);
        
        const chunks = [];
        
        ws.on('data', chunk => chunks.push(chunk));
        ws.on('error', err => reject(err));
        ws.on('end', () => {
            const data = Buffer.concat(chunks);
            resolve(data.toString('utf-8'));
        });
    });
}

6:31 - Argument of type 'string' is not assignable to parameter of type 'BufferEncoding'.


- Test

In [ ]:
{
    const filename = 'test.txt';
    
    try {
        const len = await swrite(filename, ['Hello ', '大家好']);
        console.log(`* write finish, bytes to written: ${len}`)

        const str = await sread(filename);
        console.log(`* read finish, content is: "${str}"`);
    } finally {
        await rm(filename);
    }
}

### 4.4. Watch file

- Functions

In [27]:
function sleep(ms) {
    return new Promise(resolve => {
        setTimeout(() => resolve(), ms);
    });
}

- Test

In [28]:
{
    const options = {
        recursive: true,
        encoding: 'utf-8'
    };
    
    const watcher = fs.watch('.', options, (event, filename) => {
        console.log(`* ${filename} as been ${event}`);
    });
    
    watcher.on('change', (evnet, filename) => console.log(`* ${filename} trigger ${evnet} event`));
    
    const filename = 'test.txt';
    await touch(filename);
    await fwrite(filename, Buffer.from('hello', 'utf-8'));
    await fappend(filename, Buffer.from(' world', 'utf-8'));
    
    await rm(filename);
    
    await sleep(2000);
    watcher.close();
}

7:35 - No overload matches this call.
7:35 - Overload 1 of 4, '(filename: PathLike, options: "utf-8" | { encoding?: BufferEncoding; persistent?: boolean; recursive?: boolean; } | "ascii" | "utf8" | "utf16le" | "ucs2" | "ucs-2" | "base64" | "latin1" | "binary" | "hex", listener?: (event: string, filename: string) => void): FSWatcher', gave the following error.
7:35 - Argument of type '{ recursive: boolean; encoding: string; }' is not assignable to parameter of type '"utf-8" | { encoding?: BufferEncoding; persistent?: boolean; recursive?: boolean; } | "ascii" | "utf8" | "utf16le" | "ucs2" | "ucs-2" | "base64" | "latin1" | "binary" | "hex"'.
7:35 - Type '{ recursive: boolean; encoding: string; }' is not assignable to type '{ encoding?: BufferEncoding; persistent?: boolean; recursive?: boolean; }'.
7:35 - Types of property 'encoding' are incompatible.
7:35 - Type 'string' is not assignable to type 'BufferEncoding'.
7:35 - Overload 2 of 4, '(filename: PathLike, options: { encoding: "buffer"

## 5. Glob 

- Functions

In [25]:
const glob = require('glob');

function findFiles(pattern) {
    return new Promise((resolve, reject) => {
        glob(pattern, (err, files) => {
            if (err) {
                reject(err);
            } else {
                resolve(files);
            }
        });
    });
}

- Test

In [26]:
{
    const files = await findFiles('./**/*.ipynb');
    console.log(`* found files include: ${JSON.stringify([...files])}`);
}

* found files include: ["./async.ipynb","./basic.ipynb","./collection.ipynb","./function.ipynb","./http.ipynb","./io.ipynb","./module.ipynb"]
